# Detail Rocket Deep Learning Recommendation System

## Imports and installs

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
#%pip install pandas
# %pip install seaborn
#%pip install numpy
#%pip install recsim

In [17]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import operator
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [18]:
import numpy as np
import tensorflow as tf

In [19]:
#@title Importing RecSim components
from recsim.environments import interest_evolution
from recsim.agents import full_slate_q_agent
from recsim.simulator import runner_lib

In [20]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Load Data

Preprocessing was done in EDA notebook and stored in pickle files

In [21]:
basepath = '../datasets/preprocessed_datasets/retailrocket/'
category_tree = pd.read_pickle(basepath +'category_tree.pkl')
category_tree

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
...,...,...
1664,49,1125.0
1665,1112,630.0
1666,1336,745.0
1667,689,207.0


In [22]:
events = pd.read_pickle(basepath +'events.pkl')
events

,timestamp,visitorid,event,itemid,transactionid,datetime
0,1433221332117,257597,view,355908,NaN,2015-06-02 07:02:12
1,1433224214164,992329,view,248676,NaN,2015-06-02 07:50:14
2,1433221999827,111016,view,318965,NaN,2015-06-02 07:13:19
3,1433221955914,483717,view,253185,NaN,2015-06-02 07:12:35
4,1433221337106,951259,view,367447,NaN,2015-06-02 07:02:17
...,...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN,2015-08-01 05:13:05
2756097,1438399813142,762376,view,115946,NaN,2015-08-01 05:30:13
2756098,1438397820527,1251746,view,78144,NaN,2015-08-01 04:57:00
2756099,1438398530703,1184451,view,283392,NaN,2015-08-01 05:08:50


In [23]:
item_properties = pd.read_pickle(basepath +'item_properties.pkl')
item_properties

,timestamp,itemid,property,value,datetime
0,1435460400000,460429,categoryid,1338,2015-06-28 05:00:00
1,1441508400000,206783,888,1116713 960601 n277.200,2015-09-06 05:00:00
2,1439089200000,395014,400,n552.000 639502 n720.000 424566,2015-08-09 05:00:00
3,1431226800000,59481,790,n15360.000,2015-05-10 05:00:00
4,1431831600000,156781,917,828513,2015-05-17 05:00:00
...,...,...,...,...,...
9275898,1433646000000,236931,929,n12.000,2015-06-07 05:00:00
9275899,1440903600000,455746,6,150169 639134,2015-08-30 05:00:00
9275900,1439694000000,347565,686,610834,2015-08-16 05:00:00
9275901,1433646000000,287231,867,769062,2015-06-07 05:00:00


In [24]:
visitor_actions = pd.read_pickle(basepath +'visitor_actions.pkl')
visitor_actions

,visitorid,products_viewed,num_views,num_add2cart,num_transaction
0,693516,1,1,2,0
1,829044,1,1,0,0
2,652699,1,1,0,0
3,1125936,1,1,0,0
4,1149227,1,1,0,0
...,...,...,...,...,...
1407575,998517,1,1,0,0
1407576,472345,1,1,0,0
1407577,1207677,1,1,0,0
1407578,255126,1,1,0,0


In [25]:
product_item_df = pd.read_pickle(basepath +'product_item.pkl')
product_item_df

,itemid,visitorid,view,addtocart,transaction
0,3,370720,1.0,0,0
1,3,639016,1.0,0,0
2,4,905555,1.0,0,0
3,4,1010132,1.0,0,0
4,4,1042455,1.0,0,0
...,...,...,...,...,...
2145174,466864,1388296,4.0,0,0
2145175,466867,241803,1.0,0,0
2145176,466867,450511,1.0,0,0
2145177,466867,1001859,1.0,0,0


# Training Stuff

In [26]:
tf.compat.v1.disable_eager_execution()

In [27]:
def create_agent(sess, environment, eval_mode, summary_writer=None):
    kwargs = {
        'observation_space': environment.observation_space,
        'action_space': environment.action_space,
        'summary_writer': summary_writer,
        'eval_mode': eval_mode,
    }
    return full_slate_q_agent.FullSlateQAgent(sess, **kwargs)


In [28]:
seed = 0
np.random.seed(seed)
env_config = {
    'num_candidates': 10,
    'slate_size': 2,
    'resample_documents': True,
    'seed': seed,
}

In [29]:
tmp_base_dir = '/tmp/recsim/'
runner = runner_lib.TrainRunner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_agent,
    env=interest_evolution.create_environment(env_config),
    episode_log_file="",
    max_training_steps=50,
    num_iterations=10)
runner.run_experiment()

INFO:tensorflow:max_training_steps = 50, number_iterations = 10,checkpoint frequency = 1 iterations.


INFO:tensorflow:max_training_steps = 50, number_iterations = 10,checkpoint frequency = 1 iterations.


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:max_steps_per_episode = 27000


Instructions for updating:
non-resource variables are not supported in the long term


...

INFO:tensorflow:Starting iteration 7


INFO:tensorflow:Starting iteration 8


INFO:tensorflow:Starting iteration 8


INFO:tensorflow:Starting iteration 9


INFO:tensorflow:Starting iteration 9


In [30]:
runner = runner_lib.EvalRunner(
    base_dir=tmp_base_dir,
    create_agent_fn=create_agent,
    env=interest_evolution.create_environment(env_config),
    max_eval_episodes=5,
    test_mode=True)
runner.run_experiment()

INFO:tensorflow:max_eval_episodes = 5


INFO:tensorflow:max_eval_episodes = 5


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:max_steps_per_episode = 27000
2023-03-26 17:01:03.214183: W tensorflow/c/c_api.cc:291] Operation '{name:'networks/network/Online_89/output/bias/RMSProp_2/Assign' id:34000 op device:{requested: '/device:CPU:*', assigned: ''} def:{{{node networks/network/Online_89/output/bias/RMSProp_2/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:*"](networks/network/Online_89/output/bias/RMSProp_2, networks/network/Online_89/output/bias/RMSProp_2/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


INFO:tensorflow:Beginning evaluation...


INFO:tensorflow:Beginning evaluation...


INFO:tensorflow:Restoring parameters from /tmp/recsim/train/checkpoints/tf_ckpt-9


INFO:tensorflow:Restoring parameters from /tmp/recsim/train/checkpoints/tf_ckpt-9


INFO:tensorflow:eval_file: /tmp/recsim/eval_5/returns_832


INFO:tensorflow:eval_file: /tmp/recsim/eval_5/returns_832


In [31]:
#@title Tensorboard
%tensorboard --logdir=/tmp/recsim/

Launching TensorBoard...